<a href="https://colab.research.google.com/github/ganireddikumar/Assignment1/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA PREPROCESSING**

# *Import the Libraries*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# *Importing the Dataset*

In [ ]:
data=pd.read_excel(r"D:\Crude Oil Prices Daily.xlsx")

# *Analyze the Data*

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.describe()

In [ ]:
data.info()

# *Handling Missing Data*

In [ ]:
data.isnull().any()

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(axis=0,inplace = True)

In [ ]:
data.isnull().sum()

In [ ]:
data_oil=data.reset_index()['Closing Value']

In [ ]:
data_oil

# *Feature Scaling*

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
data_oil=scaler.fit_transform(np.array(data_oil).reshape(-1,1))

# *Data Visualization*

In [ ]:
plt.plot(data_oil)

# *Splitting Data Into Train And Test*

In [ ]:
training_size=int(len(data_oil)*0.65)
test_size=len(data_oil)-training_size
train_data,test_data=data_oil[0:training_size,:],data_oil[training_size:len(data_oil),:1]

In [ ]:
training_size,test_size

In [ ]:
train_data.shape

# *Creating A Dataset With Sliding Windows*

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset (dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0] 
        dataX.append(a) 
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into x=t,t+1,t+2,t+3 and Y=t+4
time_step = 10
x_train, y_train = create_dataset(train_data, time_step) 
x_test, ytest = create_dataset(test_data, time_step)

In [ ]:
print (x_train.shape), print(y_train.shape)

In [ ]:
print (x_test.shape), print(ytest.shape)

In [ ]:
x_train

In [ ]:
x_train =x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test =x_test.reshape(x_test.shape[0],x_test.shape[1], 1)

# MODEL BUILDING

# *Importing The Model Building Libraries*

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential

# *Initializing The Model*

In [ ]:
model = Sequential()

# *Adding LSTM Layers*

In [ ]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
model.add(LSTM(50, return_sequences=True, input_shape=(10,1))) 
model.add(LSTM(50, return_sequences=True)) 
model.add(LSTM(50))

# *Adding Output Layers*

In [ ]:
model.add(Dense(1))

In [ ]:
model.summary()

# *Configure The Learning Process*

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam')

# *Train The Model*

In [ ]:
model.fit(x_train,y_train, validation_data=(x_test,ytest), epochs=50, batch_size=64, verbose=1)

# *Model Evaluation*

In [ ]:

train_predict = model.predict(x_train)
test_predict = model.predict(x_test)
train_predict=scaler.inverse_transform(train_predict) 
test_predict=scaler.inverse_transform(test_predict)

import math 
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))


# *Save The Model*

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save("crude_oil.h5")

# *Test The Model*

In [ ]:
look_back=10
trainpredictPlot = np.empty_like(data_oil)
trainpredictPlot[:, :]= np.nan
trainpredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

testPredictPlot = np.empty_like(data_oil)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(data_oil)-1, :] = test_predict

plt.plot(scaler.inverse_transform(data_oil))
plt.plot(trainpredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[2866:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
temp_input

In [ ]:
lst_output=[]
n_steps=10
i=0
while(i<10):
    if(len(temp_input)>10):
        #print(temp_input)
        x_input=np.array(temp_input[1:]) 
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1)) #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:] #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist()) 
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1

In [ ]:
day_new=np.arange(1,11) 
day_pred=np.arange(11,21)

In [ ]:
len(data_oil)

In [ ]:
plt.plot(day_new, scaler.inverse_transform(data_oil[8206:]))
plt.plot(day_pred, scaler.inverse_transform(lst_output))

In [ ]:
df3=data_oil.tolist() 
df3.extend(lst_output) 
plt.plot(df3[8100:])

In [ ]:
df3=scaler.inverse_transform(df3).tolist()

In [ ]:
plt.plot(df3)